# FISH - Pipeline - A Python interactive notebook to process FISH data

```
Author: Luis U. Aguilera
Contact Info: luis.aguilera@colostate.edu

Copyright (c) 2021 Munsky Group 
Colorado State University 
Licensed under BSD 3-Clause License.
```

### Libraries

In [ ]:
# Importing libraries
import sys
import pathlib
import warnings
import shutil
import numpy as np
import matplotlib.pyplot as plt
import tifffile

warnings.filterwarnings("ignore")

### Path to source directories

In [ ]:
# Defining directories
current_dir = pathlib.Path().absolute()
fa_dir = current_dir.parents[0].joinpath('src')

# Importing fish_analyses module
sys.path.append(str(fa_dir))
import fish_analyses as fa
from skimage.io import imread

In [ ]:
fa.Banner().print_banner()

----

# Parameters defined by the user 

----

### Paths to images

In [ ]:
# Path to credentials
desktop_path = pathlib.Path.home()/'Desktop'
path_to_config_file = desktop_path.joinpath('config.yml')
# Path to images and masks
data_folder_path =pathlib.Path('TermData/data/users/Eric/DUSP1_smFISH/20230414_DUSP1_10nM_Dex_180min_040523_60x/20230414_5')
#data_folder_path =pathlib.Path('smFISH_images/Sawyer_smFISH_images/A549/Terminator/20230425_DUSP1_A549_DEX_10min_031123')
path_to_masks_dir = None 
download_data_from_NAS = True


In [ ]:
# creating a folder to store all plots
destination_folder = pathlib.Path().absolute().joinpath('standard_format_'+data_folder_path.name)
if pathlib.Path.exists(destination_folder):
    shutil.rmtree(str(destination_folder))
    destination_folder.mkdir(parents=True, exist_ok=True)
else:
    destination_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
destination_folder

In [ ]:
local_data_dir, masks_dir, _, _, list_files_names, list_images = fa.Utilities.read_images_from_folder( path_to_config_file, data_folder_path, 
                                                    path_to_masks_dir,  download_data_from_NAS)


In [ ]:
terminator_image = list_images[0]
print(terminator_image.shape)

In [ ]:


# Visualizing the original image from the terminator scope
number_images_to_visualize = 20
_ , axes = plt.subplots(nrows=1, ncols=number_images_to_visualize, figsize=(25, 5))
for i in range (0,number_images_to_visualize ):
    img_2d = terminator_image[i,:,:]
    axes[i].imshow(img_2d, cmap='viridis') 
    axes[i].grid(False)
    axes[i].set_xticks([])
    axes[i].set_yticks([])
plt.show()

In [ ]:
# Reordering the image
number_fov = 9
number_color_channels = 2
number_z_slices = 23
number_elements_on_fov = number_color_channels*number_z_slices
y_shape, x_shape = terminator_image.shape[1], terminator_image.shape[2]

# iterating for each image. Note that the color channels are intercalated in the original image. For that reason a for loop is needed and then selecting even and odd indexes.
list_images_standard_format= []
counter=0
for i in range(number_fov):
    temp_image_fov = np.zeros((number_elements_on_fov,y_shape, x_shape))
    temp_image_fov = terminator_image[counter*number_elements_on_fov:number_elements_on_fov*(counter+1),:,:]
    temp_image = np.zeros((number_z_slices,y_shape, x_shape,number_color_channels))
    temp_image[:,:,:,0] = temp_image_fov[::2,:,:] # even indexes
    temp_image[:,:,:,1] = temp_image_fov[1::2,:,:] # odd indexes
    list_images_standard_format.append(temp_image)
    counter+=1
print(list_images_standard_format[0].shape) # [Z, Y, X, C]   

# Saving the images as tif

In [ ]:
# Saving images as tif files
for i in range(number_fov):
    image_name = 'image_'+str(i)+'.tif'
    tifffile.imsave(str(destination_folder.joinpath(image_name)), list_images_standard_format[i], metadata={'axes': 'ZYXC'})

# Visualizing the images in standard format

In [ ]:
number_images_to_visualize = list_images_standard_format[0].shape[0]
_ , axes = plt.subplots(nrows=number_fov, ncols=number_images_to_visualize, figsize=(30, 8))
for j in range (number_fov):
    for i in range (0,number_images_to_visualize ):
        axes[j,i].imshow(list_images_standard_format[j][i,:,:,0], cmap='viridis') 
        axes[j,i].grid(False)
        axes[j,i].set_xticks([])
        axes[j,i].set_yticks([])
plt.show()

In [ ]:
number_images_to_visualize = list_images_standard_format[0].shape[0]
_ , axes = plt.subplots(nrows=number_fov, ncols=number_images_to_visualize, figsize=(30, 8))
for j in range (number_fov):
    for i in range (0,number_images_to_visualize ):
        axes[j,i].imshow(list_images_standard_format[j][i,:,:,1], cmap='viridis') 
        axes[j,i].grid(False)
        axes[j,i].set_xticks([])
        axes[j,i].set_yticks([])
plt.show()